# Features

We zagen in de vorige sectie dat modellen uitdrukken hoe data tot stand komt doorheen een verzameling van wiskundige functies met twee soorten functie-argumenten:
1. _gekende_ waarden van inputvariabelen
2. _onbekende_ parameters

Die twee types staan nooit los van elkaar. **Parameters hebben altijd betrekking op één of meerdere inputvariabelen**. 

:::{note} 🌍
:icon: false
:class: simple 
In onze uitbreiding waarbij we voor de airco drempelwaardes het seizoenspatronen willen leren uit de data, hadden we het model uitgebreid van één naar vier onbekende drempelwaarden (parameters):

$$
\begin{aligned}
d_i &= f(b_w, b_l, b_z, b_h, e_i, S) \\
d_i &= \begin{cases}
b_w + e_i & \text{if } S = \text{winter} \\
b_l + e_i & \text{if } S = \text{lente} \\
b_z + e_i & \text{if } S = \text{zomer} \\
b_h + e_i & \text{if } S = \text{herfst}
\end{cases}
\end{aligned}
$$

De parameters $b_w, b_l, b_z$ en $b_h$ zijn elk gelinkt aan het feit of de observatie in een bepaald seizoen werd gemaakt. Dat wordt _gegeven_ door de input variabele $S$ die in de data aanwezig moet zijn.
:::
  
(target-features)=
Het linken van parameters aan inputvariabelen gebeurt via **features**. Deze geven **een numerieke uitdrukking aan één of meerdere inputvariabelen zodat ze wiskundig aan de overeenkomstige modelparameter(s) kunnen worden gekoppeld**.  
  
```
  +-------------------+      +-------------------+      +----------------------+
  |   Inputvariabele   | ---> |      Feature      | ---> |    Modelparameter    |
  +-------------------+      +-------------------+      +----------------------+
(bv. tijd, locatie, ...)       (numerieke waarde)          (numeriek patroon)
```
  
:::{note} 🌍
:icon: false
:class: simple 
Als we in het airco voorbeeld beschikken over een inputvariabele $S \isin \{\text{winter}, \text{lente}, \text{zomer}, \text{herfst}\}$,  moeten we die variabele op gepaste wijze vertalen naar een numerieke uitdrukking (_encoderen_) om de informatie te linken aan de parameters $b_w, b_l, b_z$ en $b_h$ van het model. We kunnen in deze context kiezen voor een zogenaamde _one-hot_ encodering.

$$
\begin{align}
\text{winter}&: \begin{bmatrix}1 & 0 & 0 & 0\end{bmatrix}, \cr
\text{lente}&: \begin{bmatrix}0 & 1 & 0 & 0\end{bmatrix}, \cr
\text{zomer}&: \begin{bmatrix}0 & 0 & 1 & 0\end{bmatrix}, \cr
\text{herfst}&: \begin{bmatrix}0 & 0 & 0 & 1\end{bmatrix}
\end{align}
$$

In plaats van 

$$
\begin{bmatrix}
\text{winter} \cr
\text{winter} \cr
\vdots \cr
\text{lente} \cr
\text{lente} \cr
\vdots \cr
\text{zomer} \cr
\text{zomer} \cr
\vdots \cr
\text{herfst} \cr
\text{herfst} \cr
\end{bmatrix}
$$

ziet de eigenlijke inputdata voor het model er dan bijvoorbeeld als volgt uit:

$$
\begin{bmatrix}
1 & 0 & 0 & 0 \cr
1 & 0 & 0 & 0 \cr
\vdots & \vdots & \vdots & \vdots \cr
0 & 1 & 0 & 0 \cr
0 & 1 & 0 & 0 \cr
\vdots & \vdots & \vdots & \vdots \cr
0 & 0 & 1 & 0 \cr
0 & 0 & 1 & 0 \cr
\vdots & \vdots & \vdots & \vdots \cr
0 & 0 & 0 & 1 \cr
0 & 0 & 0 & 1
\end{bmatrix}
$$

Dit laat toe om de seizoensinformatie als volgt aan de model parameters te linken:

$$
d_i = b_w*x_{i,1} + b_l*x_{i,2} + b_z*x_{i,3} + b_h*x_{i,4} + e_i 
$$

Voor een observatie in de zomer krijgen we dan:

$$
\begin{align}
d_{i,z} &= b_w*0 + b_l*0 + b_z*1 + b_h*0 + e_i \cr
&= b_z + e_i
\end{align}
$$
  
We hebben hier de originele inputvariabele $S$ dus omgezet in _vier binaire features_ $x_{i,1}, x_{i,2}, x_{i,3}$ en $x_{i,4}$. De eerste feature geeft aan of het datapunt (geobserveerde drempelwaarde) in de winter viel ($1$) of niet ($0$), de tweede feature geeft aan of het datapunt in de lente viel, enz. 
:::

## Feature engineering
(target-feature-engineering)=
Het proces waarbij we de link leggen tussen model parameters en de data aan de hand van features noemen we **feature engineering**.
Wanneer we zelf op maat modellen uitwerken voor bepaalde patroonherkenning, gaan we soms heel veel tijd in dit proces steken.
Er zijn vaak verschillende keuzes die gemaakt kunnen worden over hoe we precies bepaalde informatie in de data projecteren op de parameters via de features. Die keuzes kunnen bepalend zijn voor de mogelijkheid om bepaalde patronen te leren.
  

:::{note} 🌍
:icon: false
:class: simple
Stel dat we bij het airco voorbeeld enkel beschikken over [epochs](https://en.wikipedia.org/wiki/Epoch_(computing)) (milli-/seconden sinds 01/01/1970). Om tot de bovenstaande _one-hot_ encodering te komen moeten we dan volgende feature engineering stappen uitvoeren:
1. Omzetten van epochs naar seizoenscodes $S \isin \{\text{winter}, \text{lente}, \text{zomer}, \text{herfst}\}$
2. Omzetten van seizoenscodes naar de vier binaire features.  

zoals in de Python illustratie hieronder.
:::

In [ ]:
import random
import time

random.seed(42)

In [ ]:
# Create a sample of epochs
current_epoch = int(time.time())
sample_epochs = [
    current_epoch - random.randint(30 * 24 * 3600, 6 * 30 * 24 * 3600) for _ in range(10)
]
sample_epochs

[1743678853,
 1752538829,
 1753987044,
 1741965502,
 1749792428,
 1750298051,
 1750661800,
 1752065597,
 1742050718,
 1752687071]

In [31]:
from datetime import datetime

import pandas as pd


def get_season(dt):
    """
    Determine the season for a given datetime object.

    Args:
        dt (datetime): The datetime object to evaluate.

    Returns
    -------
        str: The season ("spring", "summer", "autumn", or "winter").
    """
    Y = dt.year
    if dt >= datetime(Y, 3, 21) and dt < datetime(Y, 6, 21):
        return "spring"
    elif dt >= datetime(Y, 6, 21) and dt < datetime(Y, 9, 23):
        return "summer"
    elif dt >= datetime(Y, 9, 23) and dt < datetime(Y, 12, 21):
        return "autumn"
    else:
        return "winter"


df = pd.DataFrame({"epoch": sample_epochs})
df["datetime"] = pd.to_datetime(df["epoch"], unit="s")
df["season"] = df["datetime"].apply(get_season)
df

,epoch,datetime,season
0,1743678853,2025-04-03 11:14:13,spring
1,1752538829,2025-07-15 00:20:29,summer
2,1753987044,2025-07-31 18:37:24,summer
3,1741965502,2025-03-14 15:18:22,winter
4,1749792428,2025-06-13 05:27:08,spring
5,1750298051,2025-06-19 01:54:11,spring
6,1750661800,2025-06-23 06:56:40,summer
7,1752065597,2025-07-09 12:53:17,summer
8,1742050718,2025-03-15 14:58:38,winter
9,1752687071,2025-07-16 17:31:11,summer


In [33]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(categories=[["winter", "spring", "summer", "autumn"]], sparse_output=False)
season_encoded = encoder.fit_transform(df[["season"]])
season_columns = [f"season_{cat}" for cat in encoder.categories_[0]]
season_dummies = pd.DataFrame(season_encoded, columns=season_columns, index=df.index)
df = pd.concat([df, season_dummies], axis=1)
df


,epoch,datetime,season,season_winter,season_spring,season_summer,season_autumn
0,1743678853,2025-04-03 11:14:13,spring,0.0,1.0,0.0,0.0
1,1752538829,2025-07-15 00:20:29,summer,0.0,0.0,1.0,0.0
2,1753987044,2025-07-31 18:37:24,summer,0.0,0.0,1.0,0.0
3,1741965502,2025-03-14 15:18:22,winter,1.0,0.0,0.0,0.0
4,1749792428,2025-06-13 05:27:08,spring,0.0,1.0,0.0,0.0
5,1750298051,2025-06-19 01:54:11,spring,0.0,1.0,0.0,0.0
6,1750661800,2025-06-23 06:56:40,summer,0.0,0.0,1.0,0.0
7,1752065597,2025-07-09 12:53:17,summer,0.0,0.0,1.0,0.0
8,1742050718,2025-03-15 14:58:38,winter,1.0,0.0,0.0,0.0
9,1752687071,2025-07-16 17:31:11,summer,0.0,0.0,1.0,0.0
